In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import nltk
from nltk.util import ngrams

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load your CSV file
df = pd.read_csv('/kaggle/input/english-to-hindi-nmt/Hindi_English_Truncated_Corpus - Hindi_English_Truncated_Corpus.csv')

# Split into train (70%) and temp (30%)
train, temp = train_test_split(df, test_size=0.3, random_state=42)

# Split temp into validation (10%) and test (20%) 
val, test = train_test_split(temp, test_size=2/3, random_state=42)

# Save the datasets to CSV files
train.to_csv('train.csv', index=False)
val.to_csv('validation.csv', index=False)
test.to_csv('test.csv', index=False)



In [3]:
train_data = pd.read_csv("/kaggle/working/train.csv")
test_data = pd.read_csv("/kaggle/working/test.csv")
validation_data = pd.read_csv("/kaggle/working/validation.csv")

In [4]:
train_data.head(5)

,source,english_sentence,hindi_sentence
0,tides,"Accordingly , all Financial Bills are not Mone...","तदनुसार , सभी वित्त विधेयक धन विधेयक नहीं होते ."
1,ted,"the Reverend Thomas Bayes,","रेवेरेंड थोमस बेयस,"
2,tides,Since that time the Charter has had an enormou...,उस समय से चार्टर ने अपनी सेवाओं की गुणवत्ता सु...
3,indic2012,"Rising high above feeling , a greater feeling ...",भाव से ऊपर उठकर महाभाव या प्रेम के आराध्य हैं ः-
4,indic2012,weather Nepal,वातावरण नेपाल


In [5]:
train_data.tail(5)

,source,english_sentence,hindi_sentence
27995,ted,But I borrow a lesson that I learned from him.,मगर मैने इस से कुछ सीखा है।
27996,ted,It's okay to take yourself a little less serio...,"अपने आप को कुछ कम गंभीरता से लेना ठीक है,"
27997,ted,can you be thinking about your spiritual needs...,तब ही जा कर आप अपनी आध्यातमक ज़रूरतों के बारे ...
27998,indic2012,Gandhiji is told that the failure of Pattabee ...,गाँधीजी ने पट्टाभी सितारमैय्या की हार को अपनी ...
27999,tides,"He,wanted to write about the decline and fall ...",वह ब्रिटिश साम्राज़्यवाद के पतन और विनाश के बा...


In [6]:
test_data.head(5)

,source,english_sentence,hindi_sentence
0,tides,This is particularly feasible in low-lying are...,यह शहर के आसपास के निचले हिस्सों की भराई के लि...
1,indic2012,"Tajmahal of Agra, which is tomb of Shahajahan'...",आगरे का ताजमहल शाहजहाँ की प्रिय बेगम मुमताज मह...
2,tides,Hoi agreed not to torture them collectively if...,वह एक शर्त पर मानी कि उसे प्रतिदिन एक मनुष्य भ...
3,indic2012,The project's main point of view was to join a...,इसमें सभी द्वीपों को एक जुड़े हुए द्वीप में जो...
4,tides,It is therefore important that you tell the po...,अग आप क्षतिपूर्ति चाहते हैं तो आपको पुलिस को ब...


In [7]:
test_data.tail(5)

,source,english_sentence,hindi_sentence
7995,ted,Decoration of the unfinished project,अधूरे प्रोजेक्ट की सजावट
7996,ted,"There's an axle here, of 30, 40 kg,","इस में एक एक्सल लगा है, करीब, ३० से ४० किलोग्र..."
7997,tides,"The general and dominant mood , however , is o...",वस्तुतया इसमें सामान्य और प्रभावी मनोदशा भावात...
7998,tides,Nor did she remain unlettered for long .,वह अधिक समय तक अनपढ़ भी नहीं रहीं .
7999,tides,"Dayananda Saraswati , who founded the Arya Sam...","दयानंद सरस्वती , जिन्होने सन् 1875 में ( रामलि..."


In [8]:
validation_data.head(5)

,source,english_sentence,hindi_sentence
0,indic2012,Ludhiana,लुधियाना
1,ted,of this huge number of atoms looked like.,क्वांटम समारोह क्या है
2,ted,So this is actually a piece of fig.,यह एक अंजीर का टुकड़ा है|
3,tides,"But what about jobs for the boys , you will sa...",लेकिन तब हमारे नेताओं की नौकरियों का क्या होगा...
4,indic2012,it literature/language quality was superior an...,उनकी भाषा श्रेष्ठ प्रभावी और अरबों की पसंद के ...


In [9]:
validation_data.tail(5)

,source,english_sentence,hindi_sentence
3995,ted,almost equal to that of a West European country.,पश्चिम यूरोपीय देश के लगभग बराबर है.
3996,tides,On the journey he had entrusted the suitcases ...,यात्रा के समय उन्होनें सूटकेसों को एक आस्ट्रिय...
3997,tides,That would be only the case with one to whom t...,ऐसा तो किसी बिरले के साथ ही होता होगा जिसे अल्...
3998,ted,Art is our weapon.,कला हमारा हथियार है।
3999,ted,the farmers could buy a second,"किसान दूसरे के लिये खरीद सकें,"


In [10]:
!pip install transformers torch pandas

In [11]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 4.6 MB/s eta 0:00:00


In [12]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=4215a13eaf304eb21161bc8f0bec7ad15900daac4b238a742677d588319a6ddc
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [13]:
!pip install sentencepiece

In [14]:
!pip install transformers[sentencepiece]

In [15]:
!transformers-cli cache clear

usage: transformers-cli <command> [<args>]
Transformers CLI tool: error: argument {convert,download,env,run,serve,login,whoami,logout,repo,add-new-model-like,lfs-enable-largefiles,lfs-multipart-upload,pt-to-tf}: invalid choice: 'cache' (choose from 'convert', 'download', 'env', 'run', 'serve', 'login', 'whoami', 'logout', 'repo', 'add-new-model-like', 'lfs-enable-largefiles', 'lfs-multipart-upload', 'pt-to-tf')


In [16]:
!pip install transformers[torch]

In [17]:
!pip install accelerate -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.9/330.9 kB 8.3 MB/s eta 0:00:00:00:01
  Attempting uninstall: accelerate
    Found existing installation: accelerate 0.34.2
    Uninstalling accelerate-0.34.2:
      Successfully uninstalled accelerate-0.34.2


In [18]:
!pip install git+https://github.com/csebuetnlp/normalizer

  Cloning https://github.com/csebuetnlp/normalizer to /tmp/pip-req-build-ml6kboa5
  Running command git clone --filter=blob:none --quiet https://github.com/csebuetnlp/normalizer /tmp/pip-req-build-ml6kboa5
  Resolved https://github.com/csebuetnlp/normalizer to commit d405944dde5ceeacb7c2fd3245ae2a9dea5f35c9
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 185.0/185.0 kB 4.9 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for normalizer: filename=normalizer-0.0.1-py3-none-any.whl size=6859 sha256=d328d09a5d9f42439a8b32279d26fbc0b2ec9972c92a7b9adc7d0bcea082452f
  Stored in directory: /tmp/pip-ephem-wheel-cache-5cs6_y6q/wheels/2e/79/9c/cd96d490298305d51d2da11484bb2c25fd1f759a6906708282
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186459 sha256=34f8246fabaea144e3891265917b3a453d5

In [19]:
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.6 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0


In [20]:
#!pip install transformers==4.10.3

In [21]:
!pip install torch transformers

In [22]:
#!pip install accelerate==0.20.3

In [23]:
import torch
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, DataCollatorForSeq2Seq, Trainer, TrainingArguments
from normalizer import normalize
import pandas as pd
import numpy as np
import nltk
from nltk.translate.bleu_score import corpus_bleu
from sacrebleu import sentence_bleu
from rouge_score import rouge_scorer
from sklearn.model_selection import train_test_split
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [24]:
train_data = pd.read_csv("/kaggle/working/train.csv")
test_data = pd.read_csv("/kaggle/working/test.csv")
validation_data = pd.read_csv("/kaggle/working/validation.csv")

In [25]:
train_data = train_data.dropna()
test_data = test_data.dropna()
validation_data = validation_data.dropna()

In [26]:
train_data.head()

,source,english_sentence,hindi_sentence
0,tides,"Accordingly , all Financial Bills are not Mone...","तदनुसार , सभी वित्त विधेयक धन विधेयक नहीं होते ."
1,ted,"the Reverend Thomas Bayes,","रेवेरेंड थोमस बेयस,"
2,tides,Since that time the Charter has had an enormou...,उस समय से चार्टर ने अपनी सेवाओं की गुणवत्ता सु...
3,indic2012,"Rising high above feeling , a greater feeling ...",भाव से ऊपर उठकर महाभाव या प्रेम के आराध्य हैं ः-
4,indic2012,weather Nepal,वातावरण नेपाल


In [27]:
train_data = train_data.drop(columns=['source'],axis=1)
train_data.head()

,english_sentence,hindi_sentence
0,"Accordingly , all Financial Bills are not Mone...","तदनुसार , सभी वित्त विधेयक धन विधेयक नहीं होते ."
1,"the Reverend Thomas Bayes,","रेवेरेंड थोमस बेयस,"
2,Since that time the Charter has had an enormou...,उस समय से चार्टर ने अपनी सेवाओं की गुणवत्ता सु...
3,"Rising high above feeling , a greater feeling ...",भाव से ऊपर उठकर महाभाव या प्रेम के आराध्य हैं ः-
4,weather Nepal,वातावरण नेपाल


In [28]:
test_data = test_data.drop(columns=['source'],axis=1)
test_data.head()

,english_sentence,hindi_sentence
0,This is particularly feasible in low-lying are...,यह शहर के आसपास के निचले हिस्सों की भराई के लि...
1,"Tajmahal of Agra, which is tomb of Shahajahan'...",आगरे का ताजमहल शाहजहाँ की प्रिय बेगम मुमताज मह...
2,Hoi agreed not to torture them collectively if...,वह एक शर्त पर मानी कि उसे प्रतिदिन एक मनुष्य भ...
3,The project's main point of view was to join a...,इसमें सभी द्वीपों को एक जुड़े हुए द्वीप में जो...
4,It is therefore important that you tell the po...,अग आप क्षतिपूर्ति चाहते हैं तो आपको पुलिस को ब...


In [29]:
train_data.rename(columns={'english_sentence': 'input_text', 'hindi_sentence'	: 'labels'}, inplace=True)
train_data.head()

,input_text,labels
0,"Accordingly , all Financial Bills are not Mone...","तदनुसार , सभी वित्त विधेयक धन विधेयक नहीं होते ."
1,"the Reverend Thomas Bayes,","रेवेरेंड थोमस बेयस,"
2,Since that time the Charter has had an enormou...,उस समय से चार्टर ने अपनी सेवाओं की गुणवत्ता सु...
3,"Rising high above feeling , a greater feeling ...",भाव से ऊपर उठकर महाभाव या प्रेम के आराध्य हैं ः-
4,weather Nepal,वातावरण नेपाल


In [30]:
# Rename the columns to match the expected format
test_data.rename(columns={'english_sentence': 'input_text', 'hindi_sentence'	: 'labels'}, inplace=True)
test_data.head()

,input_text,labels
0,This is particularly feasible in low-lying are...,यह शहर के आसपास के निचले हिस्सों की भराई के लि...
1,"Tajmahal of Agra, which is tomb of Shahajahan'...",आगरे का ताजमहल शाहजहाँ की प्रिय बेगम मुमताज मह...
2,Hoi agreed not to torture them collectively if...,वह एक शर्त पर मानी कि उसे प्रतिदिन एक मनुष्य भ...
3,The project's main point of view was to join a...,इसमें सभी द्वीपों को एक जुड़े हुए द्वीप में जो...
4,It is therefore important that you tell the po...,अग आप क्षतिपूर्ति चाहते हैं तो आपको पुलिस को ब...


In [31]:
validation_data.head()

,source,english_sentence,hindi_sentence
0,indic2012,Ludhiana,लुधियाना
1,ted,of this huge number of atoms looked like.,क्वांटम समारोह क्या है
2,ted,So this is actually a piece of fig.,यह एक अंजीर का टुकड़ा है|
3,tides,"But what about jobs for the boys , you will sa...",लेकिन तब हमारे नेताओं की नौकरियों का क्या होगा...
4,indic2012,it literature/language quality was superior an...,उनकी भाषा श्रेष्ठ प्रभावी और अरबों की पसंद के ...


In [32]:
validation_data = validation_data.drop(columns=['source'],axis=1)
validation_data.head()

,english_sentence,hindi_sentence
0,Ludhiana,लुधियाना
1,of this huge number of atoms looked like.,क्वांटम समारोह क्या है
2,So this is actually a piece of fig.,यह एक अंजीर का टुकड़ा है|
3,"But what about jobs for the boys , you will sa...",लेकिन तब हमारे नेताओं की नौकरियों का क्या होगा...
4,it literature/language quality was superior an...,उनकी भाषा श्रेष्ठ प्रभावी और अरबों की पसंद के ...


In [33]:
# Rename the columns to match the expected format
validation_data.rename(columns={'english_sentence': 'input_text', 'hindi_sentence'	: 'labels'}, inplace=True)
validation_data.head()

,input_text,labels
0,Ludhiana,लुधियाना
1,of this huge number of atoms looked like.,क्वांटम समारोह क्या है
2,So this is actually a piece of fig.,यह एक अंजीर का टुकड़ा है|
3,"But what about jobs for the boys , you will sa...",लेकिन तब हमारे नेताओं की नौकरियों का क्या होगा...
4,it literature/language quality was superior an...,उनकी भाषा श्रेष्ठ प्रभावी और अरबों की पसंद के ...


In [73]:
import torch
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50"
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ro_RO")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/531 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [35]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # Load the saved model
# model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_model.pt")

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Chittagong_tokenizer.json")

In [36]:
import torch
from normalizer import normalize
from transformers import MT5ForConditionalGeneration, AutoTokenizer ,DataCollatorForSeq2Seq, Trainer, TrainingArguments
import os

In [37]:
# model_name = "csebuetnlp/banglat5"
# model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)

In [38]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
'''
# Save the model
model.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'''

'\n# Save the model\nmodel.save_pretrained("/content/drive/MyDrive/movie/sylhet_model.pt")\n\n# Save the tokenizer\ntokenizer.save_pretrained("/content/drive/MyDrive/movie/sylhet_tokenizer.json")'

In [39]:
from torch.utils.data import Dataset, DataLoader

In [40]:
class Seq2SeqDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=128):
        """
        Args:
            data: A DataFrame containing 'input_text' and 'labels' columns.
            tokenizer: A Hugging Face tokenizer.
            max_length: Maximum sequence length.
        """
        self.input_text = data['input_text'].apply(normalize).tolist()
        self.labels = data['labels'].apply(normalize).tolist()
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.input_text)

    def __getitem__(self, idx):
        input_text = self.input_text[idx]
        label_text = self.labels[idx]

        # Tokenize the input text
        input_encodings = self.tokenizer(
            input_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        # Tokenize the label text to get its 'input_ids' and 'attention_mask'
        label_encodings = self.tokenizer(
            label_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': input_encodings['input_ids'].squeeze(),
            'attention_mask': input_encodings['attention_mask'].squeeze(),
            'labels': label_encodings['input_ids'].squeeze(),
        }


In [41]:
# Create train , test and validation datasets
train_dataset = Seq2SeqDataset(train_data, tokenizer)
test_dataset = Seq2SeqDataset(test_data, tokenizer)
validation_dataset = Seq2SeqDataset(validation_data, tokenizer)

# Create train , test and validation dataloaders
train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)  #batch_size=32
test_dataloader = DataLoader(test_dataset, batch_size=16) #batch_size=32
validation_dataloader = DataLoader(validation_dataset, batch_size=16) #batch_size=32


# **For training dataset**

In [42]:
# Move the model to the device (CPU or GPU)
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [43]:
# from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# # Load the saved model
# model = AutoModelForSeq2SeqLM.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_model.pt")

# # Load the tokenizer
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Movies/Mymensing_tokenizer.json")


In [44]:
#pip install transformers==4.30


In [45]:
from transformers import TrainingArguments
from transformers import AdamW

from torch.optim import AdamW

# Create a custom optimizer using torch.optim.AdamW
custom_optimizer = AdamW(
    model.parameters(),
    lr=1e-3,  # Learning rate
    eps=1e-8,  # Epsilon value to prevent division by zero
    weight_decay=0.01,  # Weight decay (L2 regularization)
)



training_args = TrainingArguments(
    output_dir='/kaggle/working/',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy="epoch",
    save_strategy="no",  # Align save strategy with evaluation strategy
#     save_total_limit=1,
#     save_steps=700,
    learning_rate=0.001,
    do_train=True,
    do_eval=True,
    remove_unused_columns=False,
    push_to_hub=False,
    report_to="none",
#     load_best_model_at_end=True,
    lr_scheduler_type="cosine_with_restarts",
    weight_decay=0.01,
    logging_dir='/kaggle/working/',
    logging_steps=10,  # Log every 10 steps,
)

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [46]:
from transformers import DataCollatorForSeq2Seq

# Create a data collator for sequence-to-sequence tasks
data_collator = DataCollatorForSeq2Seq(
    tokenizer=tokenizer,  # Your Hugging Face tokenizer
    model=model,
    padding=True,
    max_length=128,
    label_pad_token_id=tokenizer.pad_token_id,
)


In [47]:

# Define the Trainer with the custom optimizer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    optimizers=(custom_optimizer, None),  # Pass the custom optimizer here
)


# **Training start here**

In [48]:
# Fine-tune the model
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2855: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/data/data_collator.py:656: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /usr/local/src/pytorch/torch/csrc/utils/tensor_new.cpp:278.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Epoch,Training Loss,Validation Loss
1,1.011100,0.883395
2,0.827400,0.721139
3,0.691500,0.657082
4,0.628600,0.618822
5,0.563500,0.593150
6,0.494200,0.578859
7,0.580000,0.569309
8,0.555300,0.565978
9,0.532800,0.565572
10,0.564300,0.565989


TrainOutput(global_step=17500, training_loss=0.7431545500346592, metrics={'train_runtime': 14451.1113, 'train_samples_per_second': 19.375, 'train_steps_per_second': 1.211, 'total_flos': 3.70111564873728e+16, 'train_loss': 0.7431545500346592, 'epoch': 10.0})

# **Saving model and tokenizer**

In [64]:
from transformers import AutoModelForSeq2SeqLM

# Save the model
model.save_pretrained("/kaggle/working//mt5_model.pt")

# Save the tokenizer
tokenizer.save_pretrained("/kaggle/working//mt5_tokenizer.json")


('/kaggle/working//mt5_tokenizer.json/tokenizer_config.json',
 '/kaggle/working//mt5_tokenizer.json/special_tokens_map.json',
 '/kaggle/working//mt5_tokenizer.json/spiece.model',
 '/kaggle/working//mt5_tokenizer.json/added_tokens.json',
 '/kaggle/working//mt5_tokenizer.json/tokenizer.json')

# **Loading trained model and tokenizer**

In [69]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load the saved model
model = AutoModelForSeq2SeqLM.from_pretrained("/kaggle/working//mt5_model.pt")

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained("/kaggle/working//mt5_tokenizer.json")


/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:558: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [51]:
!pip install datasets

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [52]:
!pip install python-Levenshtein

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 70.5 MB/s eta 0:00:00


In [53]:
!pip install jiwer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [54]:
# Move the model to the device (CPU or GPU)
model.to(device)

MT5ForConditionalGeneration(
  (shared): Embedding(250112, 512)
  (encoder): MT5Stack(
    (embed_tokens): Embedding(250112, 512)
    (block): ModuleList(
      (0): MT5Block(
        (layer): ModuleList(
          (0): MT5LayerSelfAttention(
            (SelfAttention): MT5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): MT5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): MT5LayerFF(
            (DenseReluDense): MT5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
          

In [55]:
!pip install rouge-score
#https://github.com/google-research/google-research/tree/master/rouge
#https://huggingface.co/spaces/evaluate-metric/rouge [Different types of ROUGE scores]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


# **Loading evaluation metrics**

In [61]:
pip install nltk==3.6.7

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 34.0 MB/s eta 0:00:00
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.4
    Uninstalling nltk-3.2.4:
      Successfully uninstalled nltk-3.2.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
preprocessing 0.1.13 requires nltk==3.2.4, but you have nltk 3.6.7 which is incompatible.
textblob 0.18.0.post0 requires nltk>=3.8, but you have nltk 3.6.7 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import torch

import Levenshtein

from evaluate import load

# Define the move_to_device function

def move_to_device(batch, device):

    if isinstance(batch, torch.Tensor):

        return batch.to(device)

    elif isinstance(batch, list):

        return [move_to_device(item, device) for item in batch]

    elif isinstance(batch, dict):

        return {key: move_to_device(value, device) for key, value in batch.items()}

    else:

        return batch  # If it's not a tensor, list, or dict, leave it as is



# Load the evaluation metric for Character Error Rate (CER) and Word Error Rate (WER) and Exact Match(em)

cer_metric = load("cer")

wer_metric = load("wer")

meteor = load('meteor')

exact_match_metric = load("exact_match")



# Load BLEU and ROUGE metrics

bleu_metric = load("bleu")

rouge_metric = load('rouge')



# Initialize lists to store generated translations and references

generated_translations = []

references = []



# Generate translations for the test dataset

for batch in test_dataloader:

    # Move the batch to CUDA

    batch = move_to_device(batch, 'cuda')



    input_text = batch['input_ids']  # Access the input_text using the correct key

    labels = batch['labels']  # Access the labels using the correct key



    # Generate translations

    translation_ids = model.generate(input_text, max_length=512, num_beams=4, length_penalty=2.0, early_stopping=True)



    # Move the translation_ids to CPU to decode

    translation_ids = translation_ids.to('cpu')



    generated_translation = tokenizer.batch_decode(translation_ids, skip_special_tokens=True)



    generated_translations.extend(generated_translation)

    references.extend(tokenizer.batch_decode(labels, skip_special_tokens=True))  # Decoding the label IDs



# Make sure to move generated_translations back to CPU for evaluation if necessary

generated_translations = [translation if not isinstance(translation, str) else translation for translation in generated_translations]




In [ ]:
print("Number of generated translations:", len(generated_translations))
print("Number of references:", len(references))

In [ ]:
print(generated_translations)

In [ ]:
print(references)

In [ ]:
# Calculate Character Error Rate (CER) and Word Error Rate (WER)
results_CER = cer_metric.compute(predictions=generated_translations, references=references)
results_WER = wer_metric.compute(predictions=generated_translations, references=references)

# Calculate Exact Match (EM) and METEOR(M)
results_em = exact_match_metric.compute(predictions=generated_translations, references=references)
results_met = meteor.compute(predictions=generated_translations, references=references)

# Calculate Bilingual Evaluation Understudy (BLEU) and Recall-Oriented Understudy for Gisting Evaluation (ROUGE)
results_bleu = bleu_metric.compute(predictions=generated_translations, references=references)
results_rouge = rouge_metric.compute(predictions=generated_translations, references=references)


# Calculate Levenshtein Distance
levenshtein_distances = [Levenshtein.distance(generated, reference) for generated, reference in zip(generated_translations, references)]


# **Printing every evaluation metrics**

In [ ]:
print(results_CER)
print(results_WER)
print(results_em)
print(results_met)
print(results_bleu)
print(results_rouge)
print(levenshtein_distances)

In [ ]:
total_correct = 0
total_samples = len(references)

for generated, reference in zip(generated_translations, references):
    levenshtein_distance = Levenshtein.distance(generated, reference)
    max_length = max(len(generated), len(reference))
    accuracy = 1 - (levenshtein_distance / max_length)
    if accuracy >= 0.8:  # Adjust the threshold as needed
        total_correct += 1

accuracy = total_correct / total_samples
print("Accuracy:", accuracy)
